This code is suitable for Mamba-7B. 
Huggingface ID: TRI-ML/mamba-7b-rw

 #Settings

In [1]:
import os
import numpy as np
import random
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b") 
device=torch.device("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##import 

In [9]:
model = AutoModelForCausalLM.from_pretrained("tri-ml/mamba-7b-rw").to(dtype=torch.bfloat16).to(device)

Some weights of the model checkpoint at tri-ml/mamba-7b-rw were not used when initializing MambaForCausalLM: ['model.lm_head.weight']
- This IS expected if you are initializing MambaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MambaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from transformers import AutoTokenizer
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from mamba_ssm.utils.generation import sample
try:
    from mamba_ssm.ops.triton.selective_state_update import selective_state_update
except ImportError:
    selective_state_update = None
try:
    from mamba_ssm.ops.triton.layernorm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None
from io import BytesIO
from datasets import load_dataset
import random
import easydict
import numpy as np
import torch
import torch.nn as nn
from einops import rearrange, repeat
from mamba_ssm.ops.selective_scan_interface import selective_scan_fn, mamba_inner_fn,selective_scan_ref
try:
    from causal_conv1d import causal_conv1d_fn, causal_conv1d_update
except ImportError:
    causal_conv1d_fn, causal_conv1d_update = None
import torch.nn.functional as F

##Default settings for model 

In [12]:
model_weights = model.state_dict()
model_param="7b"

In [13]:
if model_param == "2.8b":
    layer_num = 64
    d_model = 2560
elif model_param == "130m":
    layer_num = 24
    d_model = 768

elif model_param == "7b":
    layer_num=64
    d_model=4096

d_state = 16
dt_rank = model_weights['backbone.layers.0.mixer.x_proj.weight'].shape[0] - d_state*2
d_inner = d_model * 2
d_conv=4

In [14]:
# mixer_in_proj_weights
mixer_in_proj_weights = ['backbone.layers.{0}.mixer.in_proj.weight'.format(i) for i in range(layer_num)]
mixer_out_proj_weights = ['backbone.layers.{0}.mixer.out_proj.weight'.format(i) for i in range(layer_num)]
mixer_A_log_weights=['backbone.layers.{0}.mixer.A_log'.format(i) for i in range(layer_num)]         
mixer_conv1d_weights=['backbone.layers.{0}.mixer.conv1d.weight'.format(i) for i in range (layer_num)]
mixer_conv1d_bias=['backbone.layers.{0}.mixer.conv1d.bias'.format(i) for i in range (layer_num)]
mixer_dt_proj_weights=['backbone.layers.{0}.mixer.dt_proj.weight'.format(i) for i in range(layer_num)]
mixer_dt_proj_bias=['backbone.layers.{0}.mixer.dt_proj.bias'.format(i) for i in range(layer_num)]
mixer_x_proj_weights=['backbone.layers.{0}.mixer.x_proj.weight'.format(i) for i in range(layer_num)]
norm_weights = ['backbone.layers.{0}.norm.weight'.format(i) for i in range(layer_num)]
mixer_D = ['backbone.layers.{0}.mixer.D'.format(i) for i in range(layer_num)]
norm_f='backbone.norm_f.weight'
lm_head='lm_head.weight'


#Generate a single token for multiple token inputs

In [15]:
nsamples=1
send_ssm=[]
send_conv=[]

In [16]:
def selective_scan(layer_idx,u, delta, A, B, C, D=None, z=None, delta_bias=None, delta_softplus=False,
                      return_last_state=False):
    """
    u: r(B D L)
    delta: r(B D L)
    A: c(D N) or r(D N)
    B: c(D N) or r(B N L) or r(B N 2L) or r(B G N L) or (B G N L)
    C: c(D N) or r(B N L) or r(B N 2L) or r(B G N L) or (B G N L)
    D: r(D)
    z: r(B D L)
    delta_bias: r(D), fp32

    out: r(B D L)
    last_state (optional): r(B D dstate) or c(B D dstate)
    """
    dtype_in = u.dtype
    u = u.float()
    delta = delta.float()
    if delta_bias is not None:
        delta = delta + delta_bias[..., None].float()
    if delta_softplus:
        delta = F.softplus(delta)
    batch, dim, dstate = u.shape[0], A.shape[0], A.shape[1]
    is_variable_B = B.dim() >= 3
    is_variable_C = C.dim() >= 3
    if A.is_complex():
        if is_variable_B:
            B = torch.view_as_complex(rearrange(B.float(), "... (L two) -> ... L two", two=2))
        if is_variable_C:
            C = torch.view_as_complex(rearrange(C.float(), "... (L two) -> ... L two", two=2))
    else:
        B = B.float()
        C = C.float()
    x = A.new_zeros((batch, dim, dstate))
    ys = []
    deltaA = torch.exp(torch.einsum('bdl,dn->bdln', delta, A))
    if not is_variable_B:
        deltaB_u = torch.einsum('bdl,dn,bdl->bdln', delta, B, u)
    else:
        if B.dim() == 3:
            deltaB_u = torch.einsum('bdl,bnl,bdl->bdln', delta, B, u)
        else:
            B = repeat(B, "B G N L -> B (G H) N L", H=dim // B.shape[1])
            deltaB_u = torch.einsum('bdl,bdnl,bdl->bdln', delta, B, u)
    if is_variable_C and C.dim() == 4:
        C = repeat(C, "B G N L -> B (G H) N L", H=dim // C.shape[1])
    last_state = None
    for i in range(u.shape[2]):

        x = deltaA[:, :, i] * x + deltaB_u[:, :, i] #x is as same as ssm_state(ht) [b d n]
        
        if not is_variable_C:
            y = torch.einsum('bdn,dn->bd', x, C)
        else:
            if C.dim() == 3:
                y = torch.einsum('bdn,bn->bd', x, C[:, :, i])
            else:
                y = torch.einsum('bdn,bdn->bd', x, C[:, :, :, i])
        if i == u.shape[2] - 1:
            last_state = x       

        if y.is_complex():
            y = y.real * 2
        ys.append(y)
    y = torch.stack(ys, dim=2) # (batch dim L)
    out = y if D is None else y + u * rearrange(D, "d -> d 1")
    if z is not None:
        out = out * F.silu(z)
    out = out.to(dtype=dtype_in)

    return out if not return_last_state else (out, last_state)

In [17]:
def model_inference(input_ids,seqlen,sentence):

    embedding_layer = nn.Embedding.from_pretrained(model_weights['backbone.embeddings.weight']) #or model_weights['backbone.embedding.weight'] : ~2.8b
    embedded_input = embedding_layer(input_ids)
    for layer_idx in range(layer_num):
        
        with torch.no_grad():
            if layer_idx==0:
                hidden_state=embedded_input.to(torch.bfloat16)
                prev_residual = torch.zeros(nsamples,seqlen,d_model).to(device="cuda").to(torch.bfloat16)
                ssm_state=torch.zeros(nsamples,d_model*2,d_state).to(torch.bfloat16).to(torch.bfloat16)
                conv_state=torch.zeros(nsamples,d_model*2,d_conv).to(torch.bfloat16)

            residual = hidden_state + prev_residual

            norm_cls=RMSNorm(hidden_size=d_model).cuda()
            norm_weight=model_weights[norm_weights[layer_idx]]
            norm_cls.weight = nn.Parameter(norm_weight)
            normalized_hidden_input = norm_cls(residual)

            xz = normalized_hidden_input @ model_weights[mixer_in_proj_weights[layer_idx]].T
    
            x, z = xz.chunk(2, dim=2)

            conv_state.copy_(nn.functional.pad(rearrange(x,"b l d -> b d l"), (d_conv - seqlen, 0)))
            send_conv.append(torch.clone(conv_state))
        
            x = causal_conv1d_fn(
                x=rearrange(x, "b l d -> b d l"),
                weight=rearrange(model_weights[mixer_conv1d_weights[layer_idx]], "b l d -> (b l) d"),
                bias=model_weights[mixer_conv1d_bias[layer_idx]],
                activation="silu",
            )

            
            x = rearrange(x, "b d l -> b l d")

            x_dbl = x @ model_weights[mixer_x_proj_weights[layer_idx]].T  #[b l 8192] [8192 288]

            dt, B, C = torch.split(x_dbl, [dt_rank, d_state, d_state], dim=-1)    #[b l 256]  [b l 16] [b l 16]

  
            dt = dt @ model_weights[mixer_dt_proj_weights[layer_idx]].T  #[b l 256] [256 8192]

            dt = rearrange(dt, "b l d -> b d l", l=seqlen)     #[b 8192 l]



            B = rearrange(B, "b l dstate -> b dstate l", l=seqlen).contiguous()
            C = rearrange(C, "b l dstate -> b dstate l", l=seqlen).contiguous()


            A = -torch.exp(model_weights[mixer_A_log_weights[layer_idx]]).float()

            D=model_weights[mixer_D[layer_idx]].float()
            
            dt_proj_bias=model_weights[mixer_dt_proj_bias[layer_idx]].float()

            z=rearrange(z,"b l d -> b d l")
            x=rearrange(x,"b l d -> b d l")


            y=selective_scan(
                layer_idx,
                x,
                dt,
                A,
                B,
                C,
                D,
                z=z,
                delta_bias=dt_proj_bias,
                delta_softplus=True,
                return_last_state=ssm_state is not None,
                )

            if ssm_state is not None:
                y, last_state = y
                ssm_state.copy_(last_state)

            y= rearrange(y, "b d l -> b l d")

            out = y @ model_weights[mixer_out_proj_weights[layer_idx]].T #[b l 2d] [2d d]

            hidden_state = out
            prev_residual = residual

            send_ssm.append(torch.clone(ssm_state))

    with torch.no_grad():
        residual_f=hidden_state+prev_residual
        norm_f_weight=model_weights[norm_f]
        norm_cls=RMSNorm(hidden_size=d_model).cuda()
        norm_cls.weight = nn.Parameter(norm_f_weight)
        hidden_state_f = norm_cls(residual_f)

        lm_head_weight=model_weights[lm_head]
        lm_logit=hidden_state_f @ lm_head_weight.T

        scores=[]
        scores.append(lm_logit[0][seqlen-1])

        token=sample(rearrange(scores[-1],"d -> 1 d"),top_k=40,top_p=0.9,temperature=0.9)
        gen_word=token.unsqueeze(1)
        input=tokenizer.decode(gen_word[0][0])

        return lm_logit,input


#Generate one token for one token input(Follow-up process from "Generate a single token for multiple token inputs")

In [22]:
import torch.nn
def model_gen(input_zero,gen_token_num=1024):
    gen_sentence=input_zero
    gen_token=input_zero
    act=nn.SiLU()
    for gen_idx in range(gen_token_num):
        with torch.no_grad():
            tokens = tokenizer(gen_token, return_tensors="pt")
            input_ids = tokens.input_ids.to(device="cuda")
            embedding_layer = nn.Embedding.from_pretrained(model_weights['backbone.embeddings.weight'])
            embedded_input=embedding_layer(input_ids.to(device="cuda"))
            for layer_idx in range(layer_num):
                if layer_idx==0:
                    hidden_state=embedded_input.to(torch.bfloat16)
                    prev_residual = torch.zeros(nsamples,d_model).to(device="cuda").to(torch.bfloat16)

                residual = hidden_state + prev_residual

                norm_cls=RMSNorm(hidden_size=d_model).cuda()
                norm_weight=model_weights[norm_weights[layer_idx]]
                norm_cls.weight = nn.Parameter(norm_weight)
                normalized_hidden_input = norm_cls(residual)

                xz =  normalized_hidden_input.squeeze(1) @ model_weights[mixer_in_proj_weights[layer_idx]].T
                x, z = xz.chunk(2, dim=-1)

    #----------------------conv1d stage start ----------------------------------------          
                conv_state=torch.clone(send_conv[layer_idx])

                conv_state.copy_(torch.roll(conv_state.cuda(), shifts=-1, dims=-1))  # Update state (B D W)
                conv_state[:, :, -1] = x
                x = torch.sum(conv_state.cuda() * rearrange(model_weights[mixer_conv1d_weights[layer_idx]], "d 1 w -> d w"), dim=-1)  # (B D)
                x = x + model_weights[mixer_conv1d_bias[layer_idx]]
                x = act(x)

                send_conv[layer_idx] = conv_state
    #----------------------conv1d stage end---------------------------------------- 
                x_dbl = x @ model_weights[mixer_x_proj_weights[layer_idx]].T
                dt, B, C = torch.split(x_dbl, [dt_rank, d_state, d_state], dim=-1)
            
                dt = dt @ model_weights[mixer_dt_proj_weights[layer_idx]].T
                A = -torch.exp(model_weights[mixer_A_log_weights[layer_idx]].float())
                D=model_weights[mixer_D[layer_idx]]
        
                dt_proj_bias=model_weights[mixer_dt_proj_bias[layer_idx]]

    #----------------------ssm stage start---------------------------------------- 
                ssm_state=torch.clone(send_ssm[layer_idx])

                dt = nn.functional.softplus(dt + dt_proj_bias)
                dA = torch.exp(torch.einsum("bd,dn->bdn", dt, A))
                dB = torch.einsum("bd,bn->bdn", dt, B)
                ssm_state.copy_(ssm_state.cuda() * dA + rearrange(x, "b d -> b d 1") * dB)
                y = torch.einsum("bdn,bn->bd", ssm_state.cuda(), C)
                y = y + D * x
                y = y * act(z)  # (B D)

                send_ssm[layer_idx] = ssm_state
    #----------------------ssm stage end----------------------------------------   


    #----------------------out_put----------------------------------------------            
                out = y @ model_weights[mixer_out_proj_weights[layer_idx]].T

                hidden_state = out
                prev_residual = residual


    #------------- don't modify ------------------------------------
            residual_f=hidden_state+prev_residual
            norm_f_weight=model_weights[norm_f]
            norm_cls=RMSNorm(hidden_size=d_model).cuda()
            norm_cls.weight = nn.Parameter(norm_f_weight)
            hidden_state_f = norm_cls(residual_f)

            lm_head_weight=model_weights[lm_head]
            lm_logit=hidden_state_f @ lm_head_weight.T
            token=sample(rearrange(lm_logit[0][0],"d -> 1 d"),top_k=40,top_p=0.9,temperature=0.9)
            gen_word=token.unsqueeze(1)
            gen_token=tokenizer.decode(gen_word[0][0])
            gen_sentence += gen_token

    return gen_sentence